## Group Aggregation
<br><img align="left" src="http://drive.google.com/uc?export=view&id=17lLj-fLLYk6Dxcz7yBIX7bMEl4PAESBB" width=800 height=600>

In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Sample Data
df = DataFrame({'key'  : ['A', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C'],
                'data' : [0, 5, 10, 5, 10, 15, 10, 15, 20]})
df

,key,data
0,A,0
1,B,5
2,C,10
3,A,5
4,B,10
5,C,15
6,A,10
7,B,15
8,C,20


In [3]:
#'key'를 기준으로 그룹화하여 'data'의 합을 구하기
df.groupby('key')['data'].sum()

key
A    15
B    30
C    45
Name: data, dtype: int64

In [4]:
#size()는 결측값 포함해서 빈도 측정, count()는 결측값 제외 
print(df.groupby('key')['data'].size())
print(df.groupby('key')['data'].count())

key
A    3
B    3
C    3
Name: data, dtype: int64
key
A    3
B    3
C    3
Name: data, dtype: int64


In [5]:
# When we reset the index, the old index is added as a column, and a new sequential index is used
df.groupby('key')['data'].sum().reset_index()

,key,data
0,A,15
1,B,30
2,C,45


In [6]:
#as_index를 False로 지정해주면 reset_index와 동일
df.groupby('key', as_index=False)['data'].sum()

,key,data
0,A,15
1,B,30
2,C,45


<font color = 'blue'>Iterating through groups

In [7]:
#그룹 객체는 반복문 사용가능
grouped = df.groupby('key')['data']
for name, group in grouped:
    print(name)
    print(group)

A
0     0
3     5
6    10
Name: data, dtype: int64
B
1     5
4    10
7    15
Name: data, dtype: int64
C
2    10
5    15
8    20
Name: data, dtype: int64


In [8]:
#get_group을 사용하여 특정 그룹 추출
[v for v in grouped.get_group('A')]

[0, 5, 10]

<font color = 'blue'>Filtration

In [9]:
print(df)

df.groupby('key')['data'].sum()

  key  data
0   A     0
1   B     5
2   C    10
3   A     5
4   B    10
5   C    15
6   A    10
7   B    15
8   C    20


key
A    15
B    30
C    45
Name: data, dtype: int64

In [10]:
#filter 함수를 이용하여 합이 30보다 큰 그룹 C만 추출
df.groupby('key')['data'].filter(lambda x: x.sum() > 30)

2    10
5    15
8    20
Name: data, dtype: int64

<font color = 'blue'>Using agg method

- 여러 함수를 여러 열에 적용하기 위해 agg 함수를 사용

In [11]:
#각 그룹 별 최대값 추출
df.groupby('key')['data'].agg('max')

key
A    10
B    15
C    20
Name: data, dtype: int64

In [12]:
#agg 함수 안에 리스트 형식으로 함수를 입력하면 여러 함수를 적용가능
df.groupby('key')['data'].agg(['max','min'])

,max,min
key,,
A,10,0
B,15,5
C,20,10


In [13]:
#lambda 함수 적용가능
df.groupby('key')['data'].agg(lambda x: x.max() - x.min())

key
A    10
B    10
C    10
Name: data, dtype: int64

In [14]:
#사용자 정의 함수 적용가능
def max_min(x):
    return x.max() - x.min()

df.groupby('key')['data'].agg(max_min)

key
A    10
B    10
C    10
Name: data, dtype: int64

<font color = 'blue'>Named aggregation

In [15]:
df.groupby('key')['data'].agg(['mean', 'std'])

,mean,std
key,,
A,5,5.0
B,10,5.0
C,15,5.0


In [16]:
#리스트 형식 안에 다시 괄호를 입력하면 함수(열)의 이름 변경 가능
df.groupby('key')['data'].agg([('평균','mean'),('표준편차','std')])

,평균,표준편차
key,,
A,5,5.0
B,10,5.0
C,15,5.0


<font color = 'blue'>By default, all of the numeric columns are aggregated.

In [17]:
#lambda 함수 안에 args 인자로 또 다른 매개변수 지정가능
df['average'] = df.data.apply(lambda x, y: 'above' if x > y else 'below', args=(df.data.mean(),))
df

,key,data,average
0,A,0,below
1,B,5,below
2,C,10,below
3,A,5,below
4,B,10,below
5,C,15,above
6,A,10,below
7,B,15,above
8,C,20,above


<font color = 'blue'>Applying different functions to DataFrame columns

In [18]:
#agg 함수 안에 딕셔너리 형식을 사용하면 열 별 다른 함수를 적용가능
df.groupby('key').agg({'data' : [('평균','mean'), ('합계', np.sum)], 'average' : [('범주',Series.nunique)]})

data     average
      평균  합계      범주
key                 
A      5  15       1
B     10  30       2
C     15  45       2

In [19]:
df

,key,data,average
0,A,0,below
1,B,5,below
2,C,10,below
3,A,5,below
4,B,10,below
5,C,15,above
6,A,10,below
7,B,15,above
8,C,20,above


## Pivot Tables
<br><img align="left" src="http://drive.google.com/uc?export=view&id=1HEBp4qq4GaksdQBb2fx2tRsq-gVR-j6d" width=800 height=600>

pivot table은 데이터를 그룹화하여 테이블을 재배치하고 요약된 정보를 출력
- index : 행 인덱스로 사용될 컬럼
- columns : 열 인덱스로 사용될 컬럼 (선택적으로 적용하는 것)
- value : 분석하고자 하는 컬럼
- aggfunc : 분석 시 사용할 함수(분석 메소드)

In [21]:
#Sample Data
tr = pd.DataFrame({
        'id': [1,1,1,1,2,2,2],
        'site': ['a','b','c','a','a','b','b'],
        'pageview': np.arange(1,8),
        'dwelltime': np.arange(7.0, 0, -1),
    }, columns=['id','site','pageview','dwelltime'])

tr

,id,site,pageview,dwelltime
0,1,a,1,7.0
1,1,b,2,6.0
2,1,c,3,5.0
3,1,a,4,4.0
4,2,a,5,3.0
5,2,b,6,2.0
6,2,b,7,1.0


In [28]:
'''
index : 행으로 지정할 값
columns : 열로 지정할 값
values : 그룹별 계산하고자 하는 값
aggfunc : values 값을 계산할 때 적용할 함수
fill_value : 결측값 지정
'''
pd.pivot_table(tr, index='id', columns='site', values='pageview',
              aggfunc='sum', fill_value=0)

site,a,b,c
id,,,
1,5,2,3
2,5,13,0


In [30]:
#reset_index를 사용하면 행으로 지정한 값도 열로 올라감
pd.pivot_table(tr, values='pageview', index='id', columns='site', 
               aggfunc=sum, fill_value=0).reset_index()

site,id,a,b,c
0,1,5,2,3
1,2,5,13,0


In [33]:
#.columns.name을 None으로 지정하면 열로 지정한 값의 이름을 나타내지 않음
pv = pd.pivot_table(tr, values='pageview', index='id', columns='site', 
               aggfunc=sum, fill_value=0).reset_index()

pv.columns.name = None

pv

,id,a,b,c
0,1,5,2,3
1,2,5,13,0


In [34]:
#columns를 지정하지 않아도 됨.
#aggfunc = np.mean 지정
pd.pivot_table(tr, values='pageview', index='id', 
               aggfunc=np.mean, fill_value=0).reset_index()

,id,pageview
0,1,2.5
1,2,6.0


In [35]:
#aggfunc = np.size 지정
pd.pivot_table(tr, values='pageview', columns='site', 
               aggfunc=np.size, fill_value=0)

site,a,b,c
pageview,3,3,1


## Merging DataFrames

In [36]:
#Sample Data
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data1': range(7)})
df2 = DataFrame({'key': ['a', 'b', 'd'], 'data2': range(3)})

In [37]:
display(df1)
display(df2)

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


,key,data2
0,a,0
1,b,1
2,d,2


In [40]:
#pd.merge를 사용하여 두 데이터프레임을 병합
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


<font color = 'blue'>If not specified, merge uses the overlapping column names as the keys. It’s a good practice to specify explicitly, though:

In [41]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


<font color = 'blue'>If the column names are different in each object, you can specify them separately:

In [42]:
df3 = DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data1': range(7)})
df4 = DataFrame({'rkey': ['a', 'b', 'd'], 'data2': range(3)})

In [43]:
display(df3)
display(df4)

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


,rkey,data2
0,a,0
1,b,1
2,d,2


In [44]:
#left_on, right_on 파라미터 지정
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


<font color='darkgreen'><p>
##### Merge Types
<br><img align="left" src="http://drive.google.com/uc?export=view&id=1yEU_xZ9qZrZi8IIWTMhhMIKI95XxisX0" width=900 height=800>

<font color = 'blue'>By default merge does an 'inner' join; the keys in the result are the intersection. The outer join takes the union of the keys:

In [45]:
#how 파라미터로 merge 타입을 지정할 수 있음
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


<font color = 'blue'>Pandas also provides the *merge* DataFrame method:


In [48]:
#pd.merge가 아닌, 데이터.merge로 바로 사용할 수 있음
df1.merge(df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


## Appending DataFrames
<br><img align="left" src="http://drive.google.com/uc?export=view&id=1lSpzDfbRlY_mTlJH0t1xZADJUvbHyPMw" width=800 height=600>

In [49]:
#Sample Data
df5 = DataFrame({'key': ['a', 'b', 'd'], 'data1': range(3)})
print('df1:\n', df1)
print('df5:\n',df5)

df1:
   key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   a      5
6   b      6
df5:
   key  data1
0   a      0
1   b      1
2   d      2


In [53]:
#pd.concat을 사용하여 두 데이터프레임을 병합
display(pd.concat([df1, df5]))
print('↑인덱스가 보기 불편함')

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6
0,a,0
1,b,1
2,d,2


↑인덱스가 보기 불편함


In [54]:
#1. .reset_index(drop=True)를 사용하여 인덱스 정렬
pd.concat([df1, df5]).reset_index(drop=True)

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6
7,a,0
8,b,1
9,d,2


In [55]:
#2. ignore_index 파라미터를 True로 지정하여 인덱스 정렬
pd.concat([df1, df5], ignore_index=True)

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6
7,a,0
8,b,1
9,d,2


In [56]:
#concat 에서 axis 파라미터를 1로 지정하면 두 데이터프레임을 옆으로 병합
pd.concat([df1, df5], axis=1)

,key,data1,key,data1
0,b,0,a,0.0
1,b,1,b,1.0
2,a,2,d,2.0
3,c,3,NaN,NaN
4,a,4,NaN,NaN
5,a,5,NaN,NaN
6,b,6,NaN,NaN


<font color = "blue"><p>
## Exercises

In [58]:
#Sample Data
cs = pd.read_csv('Demo.csv', encoding='cp949', engine='python')
tr = pd.read_csv('구매내역정보.csv', encoding='cp949', engine='python')

display(cs.head()) #성별 1이 남자, 2가 여자
display(tr.head())

,ID,성별,연령,거주지역
0,478207946,1,84,서울 성동구
1,479806984,1,84,서울 서초구
2,94790213,1,84,부산 사상구
3,656026338,1,84,서울 영등포구
4,433076833,1,83,경기 고양시


,ID,상품대분류명,상품중분류명,구매지역,구매일자,구매시간,구매수량,구매금액,취소여부
0,410362886,의류잡화,여성용의류-정장류,서울 송파구,20140510,1,1,198000,0
1,643279402,식품,농산물,경기 성남시,20140612,1,1,2000,0
2,643279402,식품,농산물,경기 성남시,20140612,1,1,4000,0
3,643279402,식품,농산물,경기 성남시,20140612,1,1,5000,0
4,643279402,식품,농산물,경기 성남시,20140612,1,1,9000,0


**[Exercise 1]** `60대 여성` 고객리스트를 출력하시오.

In [67]:
cs.query('성별==2 and 60<=연령<70')

,ID,성별,연령,거주지역
222,55844757,2,69,서울 서대문구
223,165639634,2,69,부산 동래구
225,969901888,2,69,서울 동대문구
226,779190802,2,69,서울 용산구
228,414448990,2,69,경기 안산시
...,...,...,...,...
3516,473266991,2,69,서울 강서구
3524,330939633,2,60,경기 안양시
3532,524493815,2,61,부산 수영구
3539,168692065,2,63,경기 안양시


**[Exercise 2]** `남성고객`과 `여성고객`은 각각 몇명인가?

In [68]:
#A1.
cs.성별.value_counts()

2    2683
1     859
Name: 성별, dtype: int64

In [70]:
#A2.
cs.groupby('성별').size()

성별
1     859
2    2683
dtype: int64

**[Exercise 3]** `여성고객`의 `평균나이`는 얼마인가?

In [72]:
cs.query('성별==2').연령.mean()

60.637718971300785

**[Exercise 4]** `성별`에 따른 `평균나이`는?

In [73]:
cs.groupby('성별').연령.mean()

성별
1    62.442375
2    60.637719
Name: 연령, dtype: float64

**[Exercise 5]** `70대 여성 고객`들은 주로 어느 `지역`에 거주하고 있는가? 상위 5개 지역만 나열하시오.

In [76]:
cs.query('성별==2 and 70<=연령<80').거주지역.value_counts().head()

경기 고양시     20
서울 송파구     13
부산 해운대구    12
경기 성남시     12
경기 안양시      9
Name: 거주지역, dtype: int64

**[Exercise 6]** (상품중분류명 기준) `Best seller`는 무엇인가?

In [79]:
tr.상품중분류명.value_counts().head(1)

가공식품    19385
Name: 상품중분류명, dtype: int64

**[Exercise 7]** `축산물`은 하루 중 언제 가장 많이 팔리는가?

In [83]:
tr.query('상품중분류명=="축산물"').구매시간.value_counts().index[0]

18

**[Exercise 8]** 전 지역에서 `판매량이 가장 많은 상품`의 `총 매출액`은 얼마인가?

In [103]:
x = tr.groupby('상품중분류명').구매수량.sum().sort_values(ascending=False).index[0]
x_take = tr.query('상품중분류명==@x').구매금액.sum()
print('판매량이 가장 많은 상품은', x, '이며, 총 매출액은', x_take, '원 입니다.')

판매량이 가장 많은 상품은 가공식품 이며, 총 매출액은 218379000 원 입니다.


**[Exercise 9]** `60대 고객`의 `총 구매액`은 얼마인가?

In [104]:
cs.head()

,ID,성별,연령,거주지역
0,478207946,1,84,서울 성동구
1,479806984,1,84,서울 서초구
2,94790213,1,84,부산 사상구
3,656026338,1,84,서울 영등포구
4,433076833,1,83,경기 고양시


In [105]:
tr.head()

,ID,상품대분류명,상품중분류명,구매지역,구매일자,구매시간,구매수량,구매금액,취소여부
0,410362886,의류잡화,여성용의류-정장류,서울 송파구,20140510,1,1,198000,0
1,643279402,식품,농산물,경기 성남시,20140612,1,1,2000,0
2,643279402,식품,농산물,경기 성남시,20140612,1,1,4000,0
3,643279402,식품,농산물,경기 성남시,20140612,1,1,5000,0
4,643279402,식품,농산물,경기 성남시,20140612,1,1,9000,0


In [107]:
pd.merge(cs, tr, on='ID').query('60<=연령<70').구매금액.sum()

2862716000

**[Exercise 10]** 각 `고객` 별로 `월` 별 `구매량`를 계산하여 출력하시오.

In [111]:
tr.head()

,ID,상품대분류명,상품중분류명,구매지역,구매일자,구매시간,구매수량,구매금액,취소여부
0,410362886,의류잡화,여성용의류-정장류,서울 송파구,20140510,1,1,198000,0
1,643279402,식품,농산물,경기 성남시,20140612,1,1,2000,0
2,643279402,식품,농산물,경기 성남시,20140612,1,1,4000,0
3,643279402,식품,농산물,경기 성남시,20140612,1,1,5000,0
4,643279402,식품,농산물,경기 성남시,20140612,1,1,9000,0


In [112]:
tr.dtypes

ID         int64
상품대분류명    object
상품중분류명    object
구매지역      object
구매일자       int64
구매시간       int64
구매수량       int64
구매금액       int64
취소여부       int64
dtype: object

In [114]:
tr['구매일자'] = tr['구매일자'].astype('str').astype('datetime64')
tr.dtypes

ID                 int64
상품대분류명            object
상품중분류명            object
구매지역              object
구매일자      datetime64[ns]
구매시간               int64
구매수량               int64
구매금액               int64
취소여부               int64
dtype: object

In [116]:
tr['구매월'] = tr.구매일자.dt.month
tr.head()

,ID,상품대분류명,상품중분류명,구매지역,구매일자,구매시간,구매수량,구매금액,취소여부,구매월
0,410362886,의류잡화,여성용의류-정장류,서울 송파구,2014-05-10,1,1,198000,0,5
1,643279402,식품,농산물,경기 성남시,2014-06-12,1,1,2000,0,6
2,643279402,식품,농산물,경기 성남시,2014-06-12,1,1,4000,0,6
3,643279402,식품,농산물,경기 성남시,2014-06-12,1,1,5000,0,6
4,643279402,식품,농산물,경기 성남시,2014-06-12,1,1,9000,0,6


cf) tr['구매월'] = tr.구매일자 //100 %100 로도 구매월을 구할 수 있음

In [119]:
pd.pivot_table(tr, index='ID', columns='구매월', values='구매수량', aggfunc='sum',
              fill_value=0)

구매월,1,2,3,4,5,6,7,8,9,10,11,12
ID,,,,,,,,,,,,
741463,1,2,3,1,0,0,0,0,0,0,0,0
909249,0,1,0,4,3,2,0,6,0,0,0,2
1065068,0,0,0,0,0,0,2,0,0,5,0,4
1310410,2,0,0,0,0,0,0,0,0,1,0,0
1387032,1,2,0,0,0,0,0,0,0,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...
998623524,0,4,1,0,4,3,0,0,3,0,1,5
999024423,0,0,0,0,1,0,0,0,0,1,1,0
999184958,3,0,2,0,7,3,0,7,0,2,0,0


**[Exercise 11]** `상품대분류` 별로 `월` 별 `판매량`를 계산하여 출력하시오.

In [122]:
pd.pivot_table(tr, index='상품대분류명', columns='구매월', values='구매수량',
              aggfunc='sum', fill_value=0)

구매월,1,2,3,4,5,6,7,8,9,10,11,12
상품대분류명,,,,,,,,,,,,
가구,65,41,51,51,39,52,47,70,54,47,51,83
가전제품,1038,466,366,465,824,846,732,1492,1553,1547,1859,1959
레포츠,537,1252,927,1288,1657,1008,1172,1021,2019,1190,1308,1050
명품,92,45,58,66,104,77,69,54,63,65,80,91
생활잡화,1056,768,785,1242,842,906,767,938,1030,1557,1827,968
식품,3442,2984,3395,4047,3909,4206,5030,5626,3742,4470,4791,4964
의류잡화,2135,2055,2171,2579,3205,2329,2677,1771,2012,2891,2529,2305
